In [ ]:
import json
from algosdk.v2client import algod
from algosdk import mnemonic,kmd,wallet
from algosdk import transaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn, wait_for_confirmation
import os 

# algod_address = "https://testnet-algorand.api.purestake.io/ps2"
# algod_token = "t5e3i9XJZ95ahVC6OIdlt7orasK0skCP4nYgt44j"
# headers = {
#    "X-API-Key": algod_token,
# }
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"


algod_client = algod.AlgodClient(algod_token, algod_address)



In [ ]:
def generate_sandbox_accounts():
    kmd_address = "http://localhost:4002"
    kmd_client = kmd.KMDClient(algod_token, kmd_address)
    wallet_name = 'unencrypted-default-wallet'
    password = ""
    walletData = wallet.Wallet(wallet_name, password, kmd_client)
    data = []
    addresses = walletData.list_keys()
    
    for addr in addresses:
        data.append({
            'sk': walletData.export_key(addr),
            'pk': addr
        })
    return data,accounts


#Local devnet accounts (auto-generated)
funded_accounts, accounts = generate_sandbox_accounts()
funded_accounts

In [ ]:
#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):    
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1       
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break

print("Account 1 address: {}".format(funded_accounts[0]['pk']))
print("Account 2 address: {}".format(funded_accounts[1]['pk']))
print("Account 3 address: {}".format(funded_accounts[2]['pk']))

In [ ]:
# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

# Account 1 creates an asset called latinum and
# sets Account 2 as the manager, reserve, freeze, and clawback address.
# Asset Creation transaction

def asset_config(_total, _unitName, _assetName):
    txn = AssetConfigTxn(
    sender=funded_accounts[1]['pk'],
    sp=params,
    total=_total,
    default_frozen=False,
    unit_name=_unitName,
    asset_name=_assetName,
    manager=funded_accounts[2]['pk'],
    reserve=funded_accounts[2]['pk'],
    freeze=funded_accounts[2]['pk'],
    clawback=funded_accounts[2]['pk'],
    url="https://path/to/my/asset/details", 
    decimals=0)
    return txn

# --Note : Length of Unit Name should not be more then 8 char

txn = asset_config(1200,"CoinDcx", "Alice'sCoinDcx")


In [ ]:
#Create Nft

#Note Do not run this Cell with asset Config function

# CREATE ASSET
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
params.fee = 1000
params.flat_fee = True

def nftConfig(_total,_unitName,_assetName):
    txn = AssetConfigTxn(sender=funded_accounts[1]['pk'],
                         sp=params,
                         total=_total,
                         default_frozen=False,
                         unit_name=_unitName,
                         asset_name=_assetName,
                         manager=funded_accounts[2]['pk'],
                         reserve=funded_accounts[2]['pk'],
                         freeze=funded_accounts[2]['pk'],
                         clawback=funded_accounts[2]['pk'],
                         url="https://path/to/my/fractional/asset/metadata.json",
                         #metadata_hash=json_metadata_hash,
                         decimals=1)
    return txn
    


nftTxn =nftConfig(10,"ALICEART","Alice's Artwork@arc3")

In [ ]:
#For Asset
# Sign with secret key of creator
stxn = txn.sign(funded_accounts[1]['sk'])

In [ ]:
#For Nft
# Sign with secret key of creator
sNFTtxn = nftTxn.sign(funded_accounts[1]['sk'])

In [ ]:
# Send the transaction to the network and retrieve the txid.
txid = algod_client.send_transaction(stxn)
print(txid)

# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client,funded_accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client,funded_accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)

In [ ]:
#For NFT ()
# Note: 

# Send the transaction to the network and retrieve the txid .
txid = algod_client.send_transaction(sNFTtxn)
print(txid)

# Retrieve the asset ID of the newly created asset by first
# ensuring that the creation transaction was confirmed,
# then grabbing the asset id from the transaction.

# Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client,funded_accounts[1]['pk'], asset_id)
    print_asset_holding(algod_client,funded_accounts[1]['pk'], asset_id)
except Exception as e:
    print(e)